# [사전작업] 챗봇 기본구조 확인 (Streamlit)

#### 필요한 라이브러리를 설치합니다. 일부 라이브러리와 호환 에러가 발생할 수 있지만, 실습과 관계 없으므로 계속 진행합니다.

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
# 사전 정의된 기본 챗봇 애플리케이션 
!cp ./basic-chat.py ../demo-app.py

# [Lab1] 지식 문서 업로드 기능 구현 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import json
import pdfplumber
from datetime import datetime
from langchain_core.documents import Document

In [ ]:
import boto3
from utils.ssm import parameter_store

region=boto3.Session().region_name
pm = parameter_store(region)

domain_endpoint = pm.get_params(key="opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"
opensearch_user_id = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="opensearch_user_password", enc=True)

In [ ]:
with open('../libs/opensearch.yml', 'r') as file:
    file_contents = file.read()

modified_contents = file_contents.replace("{opensearch_domain_endpoint}", opensearch_domain_endpoint)

with open('../libs/opensearch.yml', 'w') as file:
    file.write(modified_contents)

opensearch_domain_endpoint

### PDF 문서 처리방식 확인

문서를 어떻게 chunking 할 것인지는 RAG 성능에 많은 영향을 미칩니다.

아래 예시에서는 단순히 PDF의 Page 단위에 맞춰서 문서를 chunking 하도록 했습니다.

In [ ]:
pdf_list = ['./data/sample1_ko.pdf']

In [ ]:
# PDF에서 cid를 추출해서 ASCII 문자로 변환
def text_pruner(text, current_pdf_file):
    def replace_cid(match):
        ascii_num = int(match.group(1))
        try:
            return chr(ascii_num)
        except:
            return ''  # 변환 실패 시 빈 문자열로 처리
    cid_pattern = re.compile(r'\(cid:(\d+)\)')
    return re.sub(cid_pattern, replace_cid, text)

# PDF 파일 처리 함수
def process_pdf(pdf_file):
    print(f"Processing PDF file: {pdf_file}")
    docs = []
    source_name = pdf_file.split('/')[-1]
    type_name = source_name.split(' ')[-1].replace('.pdf', '')

    with pdfplumber.open(pdf_file) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                pruned_text = text_pruner(page_text, pdf_file)
            else:
                pruned_text = ""
            # 텍스트 길이가 20 이상인 경우에만 Documnet로 저장
            if len(pruned_text) >= 20:  
                doc = Document(
                    page_content=pruned_text.replace('\n', ' '),
                    metadata={
                        "source": source_name,
                        "type": type_name,
                        "timestamp": datetime.now().isoformat()
                    }
                )
                docs.append(doc)
    if docs:
        load_document(docs)

def load_document(docs):
    # 동작방식을 확인하기 위해, 출력만 진행 (업데이트 예정)
    print("sample doc: ", docs[0])
    print("number of docs", len(docs))    

for pdf_file in pdf_list:
    process_pdf(pdf_file)


### OpenSearch를 지식 저장소로 활용

이제 지식 저장소로 OpenSearch를 활용하기 위해, 각 Document들을 보관합니다.

#### OpenSearch 클라이언트 생성

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection

In [ ]:
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
            hosts=[
                {
                    'host': opensearch_domain_endpoint.replace("https://", ""),
                    'port': 443
                }
            ],
            http_auth=http_auth, 
            use_ssl=True,
            verify_certs=True,
            timeout=300,
            connection_class=RequestsHttpConnection
        )

#### OpenSearch에 `sample_index` 인덱스 생성

In [ ]:
with open('index_template.json', 'r') as f:
    index_body = json.load(f)

index_name = "sample_index"
exists = os_client.indices.exists(index_name)

if exists:
    os_client.indices.delete(index=index_name)
    print("Existing index has been deleted. Create new one.")
else:
    print("Index does not exist, Create one.")

os_client.indices.create(index_name, body=index_body)

In [ ]:
from botocore.config import Config

In [ ]:
retry_config = Config(
        region_name=region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
boto3_bedrock = boto3.client("bedrock-runtime", region_name=region, config=retry_config)

#### 벡터 임베딩 및 OpenSearch 벡터 저장/검색을 위한 클래스 생성

검색증강생성(RAG)의 자연어 기반 챗봇이 가능한 핵심 원리 중 하나는 벡터임베딩을 활용한 텍스트의 저장 및 검색입니다.

1. 자연어 텍스트를 벡터임베딩으로 변환해주는 Bedrock Embedding 모델을 정의하고,
2. OpenSearch에서 제공하는 벡터 저장/검색을 위한 클래스(`OpenSearchVectorSearch`)를 생성합니다.

In [ ]:
from langchain.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

In [ ]:
llmemb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-g1-text-02"
)
dimension = 1536

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llmemb,
    http_auth=http_auth,
)

#### 이제 문서 로드 함수 `load_document()`에 add_documents() 호출 구문을 추가하여, 실제 문서가 벡터화(vectorization)되어 OpenSearch에 추가되도록 합니다.

앞에서는 동작방식을 확인하기 위해, 문서를 print로만 출력하고 종료했었습니다.

In [ ]:
def load_document(docs):
    vector_db.add_documents(docs)

for pdf_file in pdf_list:
    process_pdf(pdf_file)

실행이 끝난 후에 OpenSearch에서 `sample_pdf` 인덱스를 조회해보면, 텍스트와 이 텍스트의 문맥적 의미를 담는 벡터가 함께 저장된 것이 확인됩니다.

(아래 내용은 참고만 하셔도 됩니다)

<img src="image/uploader-1.png">

# OpenSearch에서 인덱싱 결과 확인
#### ID raguser
#### PW MarsEarth1!


### 검색
```
GET sample_index/_search
{
  "query": {
    "match_all": {}
  },
  "size": 1
}
```

In [ ]:
vector_db.similarity_search("특별비용담보 특별약관에서 회사가 보상하는 비용의 범위는?")